<a href="https://colab.research.google.com/github/varmaakash/Potholes-Detection/blob/main/potholes_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Setup Google Colab Environment
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
# 2. Data Preprocessing
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/drive/MyDrive/road dataset/train'
validation_dir = '/content/drive/MyDrive/road dataset/validation'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# 3. Model Building
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 4. Model Training
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

# 5. Model Evaluation
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy*100:.2f}%')

# 6. Model Deployment
model.save('pothole_detection_model.h5')

# 7. Uploading and Predicting Images
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    if prediction[0] > 0.5:
        return 'Pothole'
    else:
        return 'Normal'



Found 681 images belonging to 2 classes.
Found 681 images belonging to 2 classes.


9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/10
21/21 [==============================] - 324s 15s/step - loss: 0.4700 - accuracy: 0.8259 - val_loss: 0.1450 - val_accuracy: 0.9732
Epoch 2/10
21/21 [==============================] - 89s 4s/step - loss: 0.1485 - accuracy: 0.9522 - val_loss: 0.3144 - val_accuracy: 0.9643
Epoch 3/10
21/21 [==============================] - 90s 4s/step - loss: 0.0809 - accuracy: 0.9738 - val_loss: 0.2451 - val_accuracy: 0.9673
Epoch 4/10
21/21 [==============================] - 88s 4s/step - loss: 0.0914 - accuracy: 0.9676 - val_loss: 0.4181 - val_accuracy: 0.9568
Epoch 5/10
21/21 [==============================] - 90s 4s/step - loss: 0.1105 - accuracy: 0.9569 - val_loss: 0.7478 - val_accuracy: 0.9107
Epoch 6/10
21/21 [==============================] - 95s 5s/step - loss: 0.0963 - accuracy: 0.9676 - val_loss: 0.6977 - val_accuracy: 0.8929
Epoch 7/10
21/21 [==============================] - 109s 5s/step - loss: 0.0919 - accuracy: 0.9

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 1s 1s/step
The image is: Normal


In [ ]:
img_path = '/poyholes.jpg'
result = predict_image(img_path)
print(f'The image is: {result}')


1/1 [==============================] - 0s 45ms/step
The image is: Pothole


In [ ]:
img_path = '/good.jpg'
result = predict_image(img_path)
print(f'The image is: {result}')


1/1 [==============================] - 0s 42ms/step
The image is: Normal


In [ ]:
img_path = '/WhatsApp Image 2024-07-16 at 15.44.46.jpeg'
result = predict_image(img_path)
print(f'The image is: {result}')

1/1 [==============================] - 0s 41ms/step
The image is: Normal
